# Lectura y visualización de Raster y Shapefiles

Crea un ambiente e instala las siguientes librerías:

``` bash
conda create -n raster_shp
conda install gdal rasterio fiona geopandas matplotlib xarray netcdf4 ipykernel

```

## Importar librerias **`Rasterio`**, **`Geopandas`** y **`Gdal`**. 

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import xarray as xr

In [ ]:
!rm dem_pros/*

## Leer raster

In [ ]:
# Abri el raster
raster_path = "./dem/Bell_dem.tif"
with rasterio.open(raster_path) as src:
    raster_data = src.read(1)  # Read the first band
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

In [ ]:
raster_data[raster_data == raster_data.min()] = 0

In [ ]:
raster_data

In [ ]:
# Plot the raster
plt.figure(figsize=(6,6))
plt.imshow(raster_data, cmap="terrain", extent=extent)
plt.colorbar(label="Elevation")
plt.title("Raster Visualization")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
# Abrir shapefile
gdf = gpd.read_file("./shp/BEL_GLA.shp")

In [ ]:
# Plot the shapefile
gdf.plot(figsize=(6,4), edgecolor="black", facecolor="none")
plt.title("Shapefile Visualization")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

# Plot raster
ax.imshow(raster_data, cmap="terrain", extent=extent)

# Plot shapefile
gdf.plot(ax=ax, edgecolor="red", facecolor="none", linewidth=1.5)

plt.title("Overlay: Raster & Shapefile")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
gdf.total_bounds

In [ ]:
import os

### Para reducir el tamaño del DEM, utilice las siguientes esquinas de latitud y longitud
longitude_upper_left = '-58.93'
latitude_upper_left = '-62.14'
longitude_lower_right = '-58.84'
latitude_lower_right = '-62.19'


### Si no desea reducir el DEM, comente lo siguiente en tres líneas
os.system('gdal_translate -projwin ' + longitude_upper_left + ' ' + latitude_upper_left + ' ' +
          longitude_lower_right + ' ' + latitude_lower_right + ' ' + './dem/Bell_dem.tif' + 
          ' ' + './dem_pros/Bell_dem_clip.tif')


In [ ]:
# Abrir el archivo raster
raster_path = "./dem_pros/Bell_dem_clip.tif"
with rasterio.open(raster_path) as src:
    raster_data = src.read(1)  # Read the first band
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

In [ ]:
raster_data[raster_data == raster_data.min()] = 0

In [ ]:
fig, ax = plt.subplots(figsize=(5,7))

# Plot raster
ax.imshow(raster_data, cmap="terrain", extent=extent)

# Plot shapefile
gdf.plot(ax=ax, edgecolor="red", facecolor="none", linewidth=1.5)

plt.title("Overlay: Raster & Shapefile")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
### Cambiar resolución de raster
aggregate_degree = '0.002'
os.system('gdalwarp -tr ' + aggregate_degree + ' ' + aggregate_degree + ' -r average ' +  
          './dem_pros/Bell_dem_clip.tif' + ' ' + './dem_pros/Bell_dem_clip_res.tif')

In [ ]:
# Abrir el archivo raster
raster_path = "./dem_pros/Bell_dem_clip_res.tif"
with rasterio.open(raster_path) as src:
    raster_data = src.read(1)  # Read the first band
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

In [ ]:
raster_data[raster_data == raster_data.min()] = 0

In [ ]:
fig, ax = plt.subplots(figsize=(5,7))

# Plot raster
cf = ax.imshow(raster_data, cmap="terrain", extent=extent)
# Plot shapefile
gdf.plot(ax=ax, edgecolor="red", facecolor="none", linewidth=1.5)

plt.title("Overlay: Raster & Shapefile")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
fig.colorbar(cf, shrink=0.85, cax=cax)

plt.show()

In [ ]:
### Convertir DEM de tif a NetCDF
os.system('gdal_translate -of NETCDF ' + "./dem_pros/Bell_dem_clip_res.tif"  + ' ' + './dem_pros/Bell_dem.nc')

In [ ]:
os.system('gdalwarp -of NETCDF  --config GDALWARP_IGNORE_BAD_CUTLINE YES -cutline ' + "./shp/BEL_GLA.shp" + ' ' + 
          "./dem_pros/Bell_dem_clip_res.tif"  + ' ' + "./dem_pros/Bell_mask.nc")

In [ ]:
dem = xr.open_dataset('./dem_pros/Bell_dem.nc').fillna(0)
array = dem.Band1.values
array[np.isnan(array)] = 0
array[array < 0] = 0
dem['Band1'][:] = array
dem.to_netcdf('./dem_pros/Bell_dem_fill.nc')

In [ ]:
### Calcular pendiente como NetCDF a partir de DEM
os.system('gdaldem slope -of NETCDF ' + './dem_pros/Bell_dem_fill.nc' + ' ' + './dem_pros/Bell_slope.nc' + ' -s 111120')

### Calcular el aspecto como NetCDF a partir de DEM
os.system('gdaldem aspect -of NETCDF ' + './dem_pros/Bell_dem_fill.nc' + ' ' + './dem_pros/Bell_aspect.nc')